## Loading Libraries

In [1]:
!pip install lightgbm

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pandas as pd
import glob
import os
from sklearn.cluster import KMeans
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import numpy as np
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import r2_score
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import  Dropout
import networkx as nx
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import chi2_contingency
import lightgbm as lgb
import networkx as nx
import warnings

In [3]:
warnings.filterwarnings('ignore')

In [4]:
df = pd.read_csv(r"C:\Users\attafuro\Desktop\EV Charging Analysis\cleaned_traffic_data.csv")

In [5]:
df. head()

,Timestamp,Station,District,Route,Direction of Travel,Lane Type,Station Length,Samples,% Observed,Total Flow,...,Lane 5 Avg Speed,Lane 6 Flow,Lane 6 Avg Occ,Lane 6 Avg Speed,Lane 7 Flow,Lane 7 Avg Occ,Lane 7 Avg Speed,Lane 8 Flow,Lane 8 Avg Occ,Lane 8 Avg Speed
0,10/01/2024 00:00:00,308512,3,50,W,ML,3.995,197,0,497.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10/01/2024 00:00:00,311831,3,5,S,OR,NaN,101,92,27.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10/01/2024 00:00:00,311832,3,5,S,FR,NaN,101,92,78.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10/01/2024 00:00:00,311844,3,5,N,OR,NaN,202,92,43.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10/01/2024 00:00:00,311847,3,5,N,OR,NaN,303,92,73.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df.shape

(4114680, 42)

## Basic Exploration 

In [7]:
# Define the final selected columns
selected_columns = [
    "Timestamp", "Station", "Route", "Direction of Travel",
    "Total Flow", "Avg Speed", "% Observed","Samples","Lane Type"
]

# Keep only the selected columns
df = df[selected_columns]

In [8]:
df.head()

,Timestamp,Station,Route,Direction of Travel,Total Flow,Avg Speed,% Observed,Samples,Lane Type
0,10/01/2024 00:00:00,308512,50,W,497.0,64.1,0,197,ML
1,10/01/2024 00:00:00,311831,5,S,27.0,NaN,92,101,OR
2,10/01/2024 00:00:00,311832,5,S,78.0,NaN,92,101,FR
3,10/01/2024 00:00:00,311844,5,N,43.0,NaN,92,202,OR
4,10/01/2024 00:00:00,311847,5,N,73.0,NaN,92,303,OR


In [9]:
df.isna().sum()

Timestamp                    0
Station                      0
Route                        0
Direction of Travel          0
Total Flow              303776
Avg Speed              1582594
% Observed                   0
Samples                      0
Lane Type                    0
dtype: int64

In [10]:
df.drop(columns=["Avg Speed"], inplace=True)

In [11]:
df.head()

,Timestamp,Station,Route,Direction of Travel,Total Flow,% Observed,Samples,Lane Type
0,10/01/2024 00:00:00,308512,50,W,497.0,0,197,ML
1,10/01/2024 00:00:00,311831,5,S,27.0,92,101,OR
2,10/01/2024 00:00:00,311832,5,S,78.0,92,101,FR
3,10/01/2024 00:00:00,311844,5,N,43.0,92,202,OR
4,10/01/2024 00:00:00,311847,5,N,73.0,92,303,OR


In [12]:
df.isna().sum()

Timestamp                   0
Station                     0
Route                       0
Direction of Travel         0
Total Flow             303776
% Observed                  0
Samples                     0
Lane Type                   0
dtype: int64

In [13]:
df = df[df['Total Flow'].notna()]

In [14]:
df.isna().sum()

Timestamp              0
Station                0
Route                  0
Direction of Travel    0
Total Flow             0
% Observed             0
Samples                0
Lane Type              0
dtype: int64

In [15]:
df.head()

,Timestamp,Station,Route,Direction of Travel,Total Flow,% Observed,Samples,Lane Type
0,10/01/2024 00:00:00,308512,50,W,497.0,0,197,ML
1,10/01/2024 00:00:00,311831,5,S,27.0,92,101,OR
2,10/01/2024 00:00:00,311832,5,S,78.0,92,101,FR
3,10/01/2024 00:00:00,311844,5,N,43.0,92,202,OR
4,10/01/2024 00:00:00,311847,5,N,73.0,92,303,OR


In [16]:
df.dtypes

Timestamp               object
Station                  int64
Route                    int64
Direction of Travel     object
Total Flow             float64
% Observed               int64
Samples                  int64
Lane Type               object
dtype: object

In [17]:
df.duplicated().sum()

0

In [18]:
df.reset_index().duplicated(subset=['Timestamp', 'Station']).sum()

0

In [19]:
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df.set_index('Timestamp', inplace=True)
df.sort_index(inplace=True)

In [20]:
# Convert Station to categorical type
df['Station'] = df['Station'].astype('category')

In [21]:
df.head()

,Station,Route,Direction of Travel,Total Flow,% Observed,Samples,Lane Type
Timestamp,,,,,,,
2024-10-01,308512,50,W,497.0,0,197,ML
2024-10-01,311831,5,S,27.0,92,101,OR
2024-10-01,311832,5,S,78.0,92,101,FR
2024-10-01,311844,5,N,43.0,92,202,OR
2024-10-01,311847,5,N,73.0,92,303,OR


## Feature Engineering

In [22]:
df['Hour'] = df.index.hour
df['Day'] = df.index.day
df['Weekday'] = df.index.weekday
df['IsWeekend'] = df['Weekday'].isin([5, 6]).astype(int)

In [23]:
for lag in range(1, 73):  # Lags 1 to 72
    df[f'TotalFlow_lag_{lag}'] = df['Total Flow'].shift(lag)

# Drop the rows that now have NaNs in these lag features (first 72 rows)
df.dropna(inplace=True)

In [24]:
df['Hour_sin'] = np.sin(2 * np.pi * df['Hour'] / 24)
df['Hour_cos'] = np.cos(2 * np.pi * df['Hour'] / 24)
df['Weekday_sin'] = np.sin(2 * np.pi * df['Weekday'] / 7)
df['Weekday_cos'] = np.cos(2 * np.pi * df['Weekday'] / 7)

In [25]:
df['Flow_roll_mean_6h'] = df['Total Flow'].rolling(window=6).mean()
df['Flow_roll_std_6h'] = df['Total Flow'].rolling(window=6).std()
df['Flow_roll_max_6h'] = df['Total Flow'].rolling(window=6).max()
df['Flow_roll_min_6h'] = df['Total Flow'].rolling(window=6).min()

In [26]:
for window in [3, 12, 24, 48, 72]:
    df[f'Flow_mean_{window}'] = df['Total Flow'].rolling(window=window).mean()
    df[f'Flow_std_{window}'] = df['Total Flow'].rolling(window=window).std()
    df[f'Flow_min_{window}'] = df['Total Flow'].rolling(window=window).min()
    df[f'Flow_max_{window}'] = df['Total Flow'].rolling(window=window).max()

In [27]:
df.dropna(inplace=True)

In [28]:
custom_holidays = [
    pd.to_datetime("2024-12-25"),
    pd.to_datetime("2024-12-31"),
    pd.to_datetime("2024-11-24"),
]
df['IsHoliday'] = df.index.to_series().apply(lambda x: 1 if x in custom_holidays else 0)

In [29]:
df.head()

,Station,Route,Direction of Travel,Total Flow,% Observed,Samples,Lane Type,Hour,Day,Weekday,...,Flow_max_24,Flow_mean_48,Flow_std_48,Flow_min_48,Flow_max_48,Flow_mean_72,Flow_std_72,Flow_min_72,Flow_max_72,IsHoliday
Timestamp,,,,,,,,,,,,,,,,,,,,,
2024-10-01,313344,50,W,190.0,92,202,ML,0,1,1,...,1068.0,395.833333,570.838627,3.0,3419.0,435.736111,522.596712,3.0,3419.0,0
2024-10-01,313349,50,W,190.0,92,202,ML,0,1,1,...,1068.0,399.458333,568.924516,3.0,3419.0,437.486111,521.544267,3.0,3419.0,0
2024-10-01,313368,80,W,787.0,100,719,ML,0,1,1,...,1068.0,404.375000,571.276906,3.0,3419.0,440.069444,522.824677,3.0,3419.0,0
2024-10-01,313386,80,W,1050.0,92,404,ML,0,1,1,...,1068.0,426.000000,575.736716,3.0,3419.0,443.555556,526.078366,3.0,3419.0,0
2024-10-01,313393,80,E,1064.0,92,505,ML,0,1,1,...,1068.0,376.937500,383.507686,3.0,1525.0,457.500000,529.066225,3.0,3419.0,0


In [30]:
for window in [12, 24, 48, 72]:
    df[f'Flow_CV_{window}'] = df[f'Flow_std_{window}'] / df[f'Flow_mean_{window}']

In [31]:
df.dropna(inplace=True)

In [32]:
# Group by station and calculate mean and std of Total Flow
station_stats = df.groupby('Station')['Total Flow'].agg(['mean', 'std'])
station_stats['CV'] = station_stats['std'] / station_stats['mean']
station_stats.dropna(inplace=True)  

## Basic Graphical Exploration

In [33]:
df['IsHoliday'] = df['IsHoliday'].astype('category')
df['Route'] = df['Route'].astype('category')

In [34]:
#  Keep only numeric features
numeric_df = df.select_dtypes(include='number')

In [35]:
cat_features = [
    'Station', 'Route', 'Direction of Travel', 'Lane Type',
    'Weekday', 'IsWeekend', 'IsHoliday'
]

for col in cat_features:
    df[col] = df[col].astype('category')

In [36]:
# Define target and features
target = 'Total Flow'
features = [col for col in df.columns if col != target]

In [37]:
# Final feature groups
flow_features = [
    'Flow_mean_3', 'Flow_std_3', 'Flow_max_3', 'Flow_min_3',
    'Flow_mean_72', 'Flow_std_72', 'Flow_std_48', 'Flow_max_24',
    'Flow_mean_24', 'Flow_mean_48',
    'Flow_roll_std_6h', 'Flow_roll_max_6h',
    'Flow_CV_48'
]

lag_features = [
    'TotalFlow_lag_1', 'TotalFlow_lag_2', 'TotalFlow_lag_3',
    'TotalFlow_lag_7', 'TotalFlow_lag_8',
    'TotalFlow_lag_15', 'TotalFlow_lag_18'
]

time_features = [
    'Hour', 'Hour_cos', 'Hour_sin', 'Weekday_cos'
]

quality_features = [
    'Samples', '% Observed'
]

# Copy original DataFrame
df_model = df.copy()

# One-hot encode low-cardinality features
df_model = pd.get_dummies(df_model, columns=['IsHoliday', 'Direction of Travel'], drop_first=True)

# Label encode high-cardinality categorical features
for col in ['Station', 'Route', 'Lane Type']:
    le = LabelEncoder()
    df_model[col] = le.fit_transform(df_model[col])

# Capture dummy columns dynamically
dummy_columns = [col for col in df_model.columns if col.startswith('IsHoliday_') or col.startswith('Direction of Travel_')]

# Final feature list
categorical_features = ['Lane Type', 'Route', 'Station']
selected_features = flow_features + lag_features + time_features + quality_features + categorical_features + dummy_columns


In [38]:
df.head()

,Station,Route,Direction of Travel,Total Flow,% Observed,Samples,Lane Type,Hour,Day,Weekday,...,Flow_max_48,Flow_mean_72,Flow_std_72,Flow_min_72,Flow_max_72,IsHoliday,Flow_CV_12,Flow_CV_24,Flow_CV_48,Flow_CV_72
Timestamp,,,,,,,,,,,,,,,,,,,,,
2024-10-01,313344,50,W,190.0,92,202,ML,0,1,1,...,3419.0,435.736111,522.596712,3.0,3419.0,0,0.581854,0.884330,1.442119,1.199342
2024-10-01,313349,50,W,190.0,92,202,ML,0,1,1,...,3419.0,437.486111,521.544267,3.0,3419.0,0,0.648914,0.895621,1.424240,1.192139
2024-10-01,313368,80,W,787.0,100,719,ML,0,1,1,...,3419.0,440.069444,522.824677,3.0,3419.0,0,0.534337,0.827111,1.412740,1.188050
2024-10-01,313386,80,W,1050.0,92,404,ML,0,1,1,...,3419.0,443.555556,526.078366,3.0,3419.0,0,0.530732,0.817180,1.351495,1.186048
2024-10-01,313393,80,E,1064.0,92,505,ML,0,1,1,...,1525.0,457.500000,529.066225,3.0,3419.0,0,0.545745,0.772216,1.017430,1.156429


In [42]:
pivot_df = df.pivot(columns='Station', values='Total Flow')

pivot_df = pivot_df.sort_index()

In [43]:
missing_summary = pivot_df.isnull().sum()
print("Missing values per station:\n", missing_summary)

Missing values per station:
 Station
308511     3
308512     1
311831     2
311832     2
311844     2
          ..
3423094    0
3900021    0
3900022    0
3900023    0
3900024    0
Length: 1806, dtype: int64


In [46]:
pivot_df = pivot_df.fillna(method='ffill')       # Forward fill
pivot_df = pivot_df.fillna(method='bfill')       # Backward fill
pivot_df = pivot_df.interpolate(method='time')   # Time-based interpolation


display(pivot_df.head())

Station,308511,308512,311831,311832,311844,311847,311903,311930,311973,311974,...,3423063,3423064,3423065,3423066,3423091,3423094,3900021,3900022,3900023,3900024
Timestamp,,,,,,,,,,,,,,,,,,,,,
2024-10-01 00:00:00,12.0,489.0,24.0,44.0,25.0,59.0,1085.0,429.0,23.0,725.0,...,2.0,2632.0,2.0,3.0,75.0,0.0,556.0,320.0,546.0,320.0
2024-10-01 01:00:00,12.0,489.0,24.0,44.0,25.0,59.0,1085.0,429.0,23.0,725.0,...,3.0,2824.0,1.0,3.0,47.0,20.0,520.0,225.0,505.0,225.0
2024-10-01 02:00:00,12.0,473.0,21.0,26.0,40.0,28.0,960.0,321.0,15.0,611.0,...,3.0,3184.0,4.0,3.0,43.0,27.0,482.0,201.0,519.0,201.0
2024-10-01 03:00:00,20.0,503.0,34.0,27.0,68.0,63.0,988.0,451.0,10.0,749.0,...,2.0,3263.0,1.0,3.0,61.0,15.0,522.0,246.0,583.0,246.0
2024-10-01 04:00:00,55.0,687.0,56.0,44.0,121.0,87.0,1325.0,1059.0,24.0,1514.0,...,18.0,3654.0,6.0,3.0,153.0,79.0,649.0,446.0,915.0,446.0


In [50]:
# Check how many missing values per station
missing_per_station = pivot_df.isnull().sum()

# Drop stations with too many missing values (e.g., more than 10% of rows)
threshold = len(pivot_df) * 0.1
stations_to_keep = missing_per_station[missing_per_station <= threshold].index
pivot_df = pivot_df[stations_to_keep]

# Fill remaining missing values using time-based interpolation
pivot_df = pivot_df.interpolate(method='time')

# Drop any remaining NaNs just in case interpolation missed anything (e.g. first row)
pivot_df = pivot_df.dropna()

# Confirm final shape
print("Cleaned pivot shape:", pivot_df.shape

Cleaned pivot shape: (2208, 1806)


In [83]:
# Define function to generate input-output pairs per station using final flow value
def generate_multistation_dataset(data, input_length, forecast_horizon):
    X, Y = [], []
    for t in range(len(data) - input_length - forecast_horizon + 1):
        x_window = data[t : t + input_length]
        y_window = data[t + input_length : t + input_length + forecast_horizon]

        X.append(x_window)                     # shape: (input_length, num_stations)
        Y.append(y_window[-1])                 # predict flow at final timestep of horizon

    return np.array(X), np.array(Y)

# Convert pivot_df to a NumPy array
data = pivot_df.values

# Define input-output configurations for each forecast horizon
horizon_configs = {
    "12h": (12, 12),
    "24h": (24, 24),
    "48h": (48, 48),
    "72h": (72, 72),
}

# Generate datasets for each horizon
datasets = {}
for label, (input_len, forecast_horizon) in horizon_configs.items():
    X, Y = generate_multistation_dataset(data, input_len, forecast_horizon)
    datasets[label] = (X, Y)
    print(f"{label} -> X shape: {X.shape}, Y shape: {Y.shape}")


12h -> X shape: (2185, 12, 1806), Y shape: (2185, 1806)
24h -> X shape: (2161, 24, 1806), Y shape: (2161, 1806)
48h -> X shape: (2113, 48, 1806), Y shape: (2113, 1806)
72h -> X shape: (2065, 72, 1806), Y shape: (2065, 1806)


In [84]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# === Step 1: Unpack 12h dataset ===
X_12, Y_12 = datasets["12h"]

# === Step 2: Normalize per station ===
def normalize_data_per_station(X, Y):
    num_samples, time_steps, num_stations = X.shape
    X_reshaped = X.reshape(-1, num_stations)  # (samples * time_steps, stations)

    # Scale X
    X_scaler = MinMaxScaler()
    X_scaled = X_scaler.fit_transform(X_reshaped).reshape(num_samples, time_steps, num_stations)

    # Scale Y
    Y_scaler = MinMaxScaler()
    Y_scaled = Y_scaler.fit_transform(Y)

    return X_scaled, Y_scaled, X_scaler, Y_scaler

X_12_scaled, Y_12_scaled, X_scaler_12, Y_scaler_12 = normalize_data_per_station(X_12, Y_12)

# === Step 3: Train/Val/Test split (70/15/15) ===
# Split 70% train, 30% temp
X_train_12, X_temp_12, Y_train_12, Y_temp_12 = train_test_split(
    X_12_scaled, Y_12_scaled, test_size=0.3, random_state=42
)

# Split temp into 15% val and 15% test
X_val_12, X_test_12, Y_val_12, Y_test_12 = train_test_split(
    X_temp_12, Y_temp_12, test_size=0.5, random_state=42
)

# === Confirm final shapes ===
print("Final Splits (12h):")
print(f"Train: {X_train_12.shape}, {Y_train_12.shape}")
print(f"Val:   {X_val_12.shape}, {Y_val_12.shape}")
print(f"Test:  {X_test_12.shape}, {Y_test_12.shape}")

Final Splits (12h):
Train: (1529, 12, 1806), (1529, 1806)
Val:   (328, 12, 1806), (328, 1806)
Test:  (328, 12, 1806), (328, 1806)


In [85]:
import torch
import torch.nn as nn

# === Define Model ===
class CNN_GRU_LSTM(nn.Module):
    def __init__(self, num_stations, cnn_filters=64, gru_hidden=64, lstm_hidden=64, dropout=0.3):
        super(CNN_GRU_LSTM, self).__init__()
        self.num_stations = num_stations

        # CNN Block
        self.cnn = nn.Sequential(
            nn.Conv1d(in_channels=num_stations, out_channels=cnn_filters, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv1d(in_channels=cnn_filters, out_channels=cnn_filters, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Dropout(dropout)
        )

        # GRU Block
        self.gru = nn.GRU(input_size=cnn_filters, hidden_size=gru_hidden, num_layers=2,
                          batch_first=True, dropout=dropout)

        # LSTM Block
        self.lstm = nn.LSTM(input_size=gru_hidden, hidden_size=lstm_hidden, num_layers=2,
                            batch_first=True, dropout=dropout)

        # Fully Connected Layer
        self.fc = nn.Sequential(
            nn.Linear(lstm_hidden, 512),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(512, num_stations)  # Final output: flow per station
        )

    def forward(self, x):
        x = x.permute(0, 2, 1)  # (batch, stations, timesteps)
        x = self.cnn(x)
        x = x.permute(0, 2, 1)  # (batch, timesteps, filters)

        out_gru, _ = self.gru(x)
        out_lstm, _ = self.lstm(out_gru)

        out = self.fc(out_lstm[:, -1, :])  # use only last time step
        return out

In [86]:
from torch.utils.data import DataLoader, TensorDataset

# === Prepare tensors ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

X_train_tensor = torch.tensor(X_train_12, dtype=torch.float32).to(device)
Y_train_tensor = torch.tensor(Y_train_12, dtype=torch.float32).to(device)
X_val_tensor = torch.tensor(X_val_12, dtype=torch.float32).to(device)
Y_val_tensor = torch.tensor(Y_val_12, dtype=torch.float32).to(device)

# === Dataloaders ===
train_loader = DataLoader(TensorDataset(X_train_tensor, Y_train_tensor), batch_size=64, shuffle=True)
val_loader = DataLoader(TensorDataset(X_val_tensor, Y_val_tensor), batch_size=64)

# === Model ===
model = CNN_GRU_LSTM(num_stations=Y_train_12.shape[1]).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.SmoothL1Loss()  # Huber loss

# === Training Loop ===
for epoch in range(20):
    model.train()
    train_loss = 0
    for xb, yb in train_loader:
        optimizer.zero_grad()
        pred = model(xb)
        loss = loss_fn(pred, yb)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    print(f"Epoch {epoch+1}/20 - Train Loss: {train_loss / len(train_loader):.4f}")


Epoch 1/20 - Train Loss: 0.0443
Epoch 2/20 - Train Loss: 0.0184
Epoch 3/20 - Train Loss: 0.0116
Epoch 4/20 - Train Loss: 0.0097
Epoch 5/20 - Train Loss: 0.0087
Epoch 6/20 - Train Loss: 0.0080
Epoch 7/20 - Train Loss: 0.0079
Epoch 8/20 - Train Loss: 0.0073
Epoch 9/20 - Train Loss: 0.0072
Epoch 10/20 - Train Loss: 0.0070
Epoch 11/20 - Train Loss: 0.0067
Epoch 12/20 - Train Loss: 0.0064
Epoch 13/20 - Train Loss: 0.0062
Epoch 14/20 - Train Loss: 0.0060
Epoch 15/20 - Train Loss: 0.0059
Epoch 16/20 - Train Loss: 0.0059
Epoch 17/20 - Train Loss: 0.0055
Epoch 18/20 - Train Loss: 0.0053
Epoch 19/20 - Train Loss: 0.0051
Epoch 20/20 - Train Loss: 0.0056


In [87]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt

# Prepare test tensors 
X_test_tensor = torch.tensor(X_test_12, dtype=torch.float32).to(device)

# === Evaluate ===
model.eval()
with torch.no_grad():
    y_pred_scaled = model(X_test_tensor).cpu().numpy()
    y_true_scaled = Y_test_12  # Already on CPU and scaled

# === Inverse transform ===
y_pred_inv = Y_scaler_12.inverse_transform(y_pred_scaled)
y_true_inv = Y_scaler_12.inverse_transform(y_true_scaled)

# === Compute Metrics ===
rmse = np.sqrt(mean_squared_error(y_true_inv, y_pred_inv))
mae = mean_absolute_error(y_true_inv, y_pred_inv)

print("Evaluation Metrics for 12h Horizon")
print(f"RMSE: {rmse:.2f}")
print(f"MAE:  {mae:.2f}")



Evaluation Metrics for 12h Horizon
RMSE: 298.16
MAE:  147.36


In [89]:
# ===  Unpack 24h ===
X_24, Y_24 = datasets["24h"]

# ===  Normalize per station ===
X_24_scaled, Y_24_scaled, X_scaler_24, Y_scaler_24 = normalize_data_per_station(X_24, Y_24)

# === Train/Val/Test split (70/15/15) ===
X_train_24, X_temp_24, Y_train_24, Y_temp_24 = train_test_split(X_24_scaled, Y_24_scaled, test_size=0.3, random_state=42)
X_val_24, X_test_24, Y_val_24, Y_test_24 = train_test_split(X_temp_24, Y_temp_24, test_size=0.5, random_state=42)

print("Final Splits (24h):")
print(f"Train: {X_train_24.shape}, {Y_train_24.shape}")
print(f"Val:   {X_val_24.shape}, {Y_val_24.shape}")
print(f"Test:  {X_test_24.shape}, {Y_test_24.shape}")


Final Splits (24h):
Train: (1512, 24, 1806), (1512, 1806)
Val:   (324, 24, 1806), (324, 1806)
Test:  (325, 24, 1806), (325, 1806)


In [90]:
# === Prepare tensors ===
X_train_tensor = torch.tensor(X_train_24, dtype=torch.float32).to(device)
Y_train_tensor = torch.tensor(Y_train_24, dtype=torch.float32).to(device)
X_val_tensor = torch.tensor(X_val_24, dtype=torch.float32).to(device)
Y_val_tensor = torch.tensor(Y_val_24, dtype=torch.float32).to(device)

# === Dataloaders ===
train_loader = DataLoader(TensorDataset(X_train_tensor, Y_train_tensor), batch_size=64, shuffle=True)
val_loader = DataLoader(TensorDataset(X_val_tensor, Y_val_tensor), batch_size=64)


In [91]:
# === New model instance for 24h ===
model = CNN_GRU_LSTM(num_stations=Y_train_24.shape[1]).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.SmoothL1Loss()  # Huber loss again

# === Training Loop ===
for epoch in range(20):
    model.train()
    train_loss = 0
    for xb, yb in train_loader:
        optimizer.zero_grad()
        pred = model(xb)
        loss = loss_fn(pred, yb)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    print(f"Epoch {epoch+1}/20 - Train Loss: {train_loss / len(train_loader):.4f}")


Epoch 1/20 - Train Loss: 0.0440
Epoch 2/20 - Train Loss: 0.0154
Epoch 3/20 - Train Loss: 0.0090
Epoch 4/20 - Train Loss: 0.0081
Epoch 5/20 - Train Loss: 0.0076
Epoch 6/20 - Train Loss: 0.0076
Epoch 7/20 - Train Loss: 0.0069
Epoch 8/20 - Train Loss: 0.0067
Epoch 9/20 - Train Loss: 0.0063
Epoch 10/20 - Train Loss: 0.0062
Epoch 11/20 - Train Loss: 0.0058
Epoch 12/20 - Train Loss: 0.0055
Epoch 13/20 - Train Loss: 0.0056
Epoch 14/20 - Train Loss: 0.0052
Epoch 15/20 - Train Loss: 0.0051
Epoch 16/20 - Train Loss: 0.0049
Epoch 17/20 - Train Loss: 0.0047
Epoch 18/20 - Train Loss: 0.0046
Epoch 19/20 - Train Loss: 0.0045
Epoch 20/20 - Train Loss: 0.0043


In [92]:
# === Evaluate on 24h test set ===
X_test_tensor = torch.tensor(X_test_24, dtype=torch.float32).to(device)

model.eval()
with torch.no_grad():
    y_pred_scaled = model(X_test_tensor).cpu().numpy()
    y_true_scaled = Y_test_24

y_pred_inv = Y_scaler_24.inverse_transform(y_pred_scaled)
y_true_inv = Y_scaler_24.inverse_transform(y_true_scaled)

# === Metrics ===
rmse = np.sqrt(mean_squared_error(y_true_inv, y_pred_inv))
mae = mean_absolute_error(y_true_inv, y_pred_inv)

print("Evaluation Metrics for 24h Horizon")
print(f"RMSE: {rmse:.2f}")
print(f"MAE:  {mae:.2f}")


Evaluation Metrics for 24h Horizon
RMSE: 225.50
MAE:  112.40


In [93]:
# === Unpack 48h data ===
X_48, Y_48 = datasets["48h"]

# === Normalize per station ===
X_48_scaled, Y_48_scaled, X_scaler_48, Y_scaler_48 = normalize_data_per_station(X_48, Y_48)

# === Split into Train / Val / Test ===
X_train_48, X_temp_48, Y_train_48, Y_temp_48 = train_test_split(X_48_scaled, Y_48_scaled, test_size=0.3, random_state=42)
X_val_48, X_test_48, Y_val_48, Y_test_48 = train_test_split(X_temp_48, Y_temp_48, test_size=0.5, random_state=42)

# === Confirm ===
print("Final Splits (48h):")
print(f"Train: {X_train_48.shape}, {Y_train_48.shape}")
print(f"Val:   {X_val_48.shape}, {Y_val_48.shape}")
print(f"Test:  {X_test_48.shape}, {Y_test_48.shape}")

Final Splits (48h):
Train: (1479, 48, 1806), (1479, 1806)
Val:   (317, 48, 1806), (317, 1806)
Test:  (317, 48, 1806), (317, 1806)


In [94]:
X_train_tensor = torch.tensor(X_train_48, dtype=torch.float32).to(device)
Y_train_tensor = torch.tensor(Y_train_48, dtype=torch.float32).to(device)
X_val_tensor = torch.tensor(X_val_48, dtype=torch.float32).to(device)
Y_val_tensor = torch.tensor(Y_val_48, dtype=torch.float32).to(device)

train_loader = DataLoader(TensorDataset(X_train_tensor, Y_train_tensor), batch_size=64, shuffle=True)
val_loader = DataLoader(TensorDataset(X_val_tensor, Y_val_tensor), batch_size=64)

In [95]:
model = CNN_GRU_LSTM(num_stations=Y_train_48.shape[1]).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.SmoothL1Loss()

for epoch in range(20):
    model.train()
    train_loss = 0
    for xb, yb in train_loader:
        optimizer.zero_grad()
        pred = model(xb)
        loss = loss_fn(pred, yb)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    print(f"Epoch {epoch+1}/20 - Train Loss: {train_loss / len(train_loader):.4f}")

Epoch 1/20 - Train Loss: 0.0425
Epoch 2/20 - Train Loss: 0.0150
Epoch 3/20 - Train Loss: 0.0104
Epoch 4/20 - Train Loss: 0.0090
Epoch 5/20 - Train Loss: 0.0084
Epoch 6/20 - Train Loss: 0.0078
Epoch 7/20 - Train Loss: 0.0079
Epoch 8/20 - Train Loss: 0.0074
Epoch 9/20 - Train Loss: 0.0073
Epoch 10/20 - Train Loss: 0.0071
Epoch 11/20 - Train Loss: 0.0074
Epoch 12/20 - Train Loss: 0.0065
Epoch 13/20 - Train Loss: 0.0063
Epoch 14/20 - Train Loss: 0.0065
Epoch 15/20 - Train Loss: 0.0060
Epoch 16/20 - Train Loss: 0.0060
Epoch 17/20 - Train Loss: 0.0064
Epoch 18/20 - Train Loss: 0.0057
Epoch 19/20 - Train Loss: 0.0059
Epoch 20/20 - Train Loss: 0.0054


In [96]:
X_test_tensor = torch.tensor(X_test_48, dtype=torch.float32).to(device)

model.eval()
with torch.no_grad():
    y_pred_scaled = model(X_test_tensor).cpu().numpy()
    y_true_scaled = Y_test_48

y_pred_inv = Y_scaler_48.inverse_transform(y_pred_scaled)
y_true_inv = Y_scaler_48.inverse_transform(y_true_scaled)

rmse = np.sqrt(mean_squared_error(y_true_inv, y_pred_inv))
mae = mean_absolute_error(y_true_inv, y_pred_inv)

print("Evaluation Metrics for 48h Horizon")
print(f"RMSE: {rmse:.2f}")
print(f"MAE:  {mae:.2f}")

Evaluation Metrics for 48h Horizon
RMSE: 272.79
MAE:  142.87


In [97]:
# === Unpack 72h data ===
X_72, Y_72 = datasets["72h"]

# === Normalize per station ===
X_72_scaled, Y_72_scaled, X_scaler_72, Y_scaler_72 = normalize_data_per_station(X_72, Y_72)

# === Split into Train / Val / Test ===
X_train_72, X_temp_72, Y_train_72, Y_temp_72 = train_test_split(X_72_scaled, Y_72_scaled, test_size=0.3, random_state=42)
X_val_72, X_test_72, Y_val_72, Y_test_72 = train_test_split(X_temp_72, Y_temp_72, test_size=0.5, random_state=42)

print("Final Splits (72h):")
print(f"Train: {X_train_72.shape}, {Y_train_72.shape}")
print(f"Val:   {X_val_72.shape}, {Y_val_72.shape}")
print(f"Test:  {X_test_72.shape}, {Y_test_72.shape}")


Final Splits (72h):
Train: (1445, 72, 1806), (1445, 1806)
Val:   (310, 72, 1806), (310, 1806)
Test:  (310, 72, 1806), (310, 1806)


In [98]:
X_train_tensor = torch.tensor(X_train_72, dtype=torch.float32).to(device)
Y_train_tensor = torch.tensor(Y_train_72, dtype=torch.float32).to(device)
X_val_tensor = torch.tensor(X_val_72, dtype=torch.float32).to(device)
Y_val_tensor = torch.tensor(Y_val_72, dtype=torch.float32).to(device)

train_loader = DataLoader(TensorDataset(X_train_tensor, Y_train_tensor), batch_size=64, shuffle=True)
val_loader = DataLoader(TensorDataset(X_val_tensor, Y_val_tensor), batch_size=64)

In [99]:
model = CNN_GRU_LSTM(num_stations=Y_train_72.shape[1]).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.SmoothL1Loss()

for epoch in range(20):
    model.train()
    train_loss = 0
    for xb, yb in train_loader:
        optimizer.zero_grad()
        pred = model(xb)
        loss = loss_fn(pred, yb)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    print(f"Epoch {epoch+1}/20 - Train Loss: {train_loss / len(train_loader):.4f}")

Epoch 1/20 - Train Loss: 0.0445
Epoch 2/20 - Train Loss: 0.0155
Epoch 3/20 - Train Loss: 0.0105
Epoch 4/20 - Train Loss: 0.0097
Epoch 5/20 - Train Loss: 0.0087
Epoch 6/20 - Train Loss: 0.0081
Epoch 7/20 - Train Loss: 0.0079
Epoch 8/20 - Train Loss: 0.0074
Epoch 9/20 - Train Loss: 0.0074
Epoch 10/20 - Train Loss: 0.0071
Epoch 11/20 - Train Loss: 0.0066
Epoch 12/20 - Train Loss: 0.0065
Epoch 13/20 - Train Loss: 0.0062
Epoch 14/20 - Train Loss: 0.0063
Epoch 15/20 - Train Loss: 0.0059
Epoch 16/20 - Train Loss: 0.0062
Epoch 17/20 - Train Loss: 0.0059
Epoch 18/20 - Train Loss: 0.0053
Epoch 19/20 - Train Loss: 0.0051
Epoch 20/20 - Train Loss: 0.0051


In [100]:
X_test_tensor = torch.tensor(X_test_72, dtype=torch.float32).to(device)

model.eval()
with torch.no_grad():
    y_pred_scaled = model(X_test_tensor).cpu().numpy()
    y_true_scaled = Y_test_72

y_pred_inv = Y_scaler_72.inverse_transform(y_pred_scaled)
y_true_inv = Y_scaler_72.inverse_transform(y_true_scaled)

rmse = np.sqrt(mean_squared_error(y_true_inv, y_pred_inv))
mae = mean_absolute_error(y_true_inv, y_pred_inv)

print("Evaluation Metrics for 72h Horizon")
print(f"RMSE: {rmse:.2f}")
print(f"MAE:  {mae:.2f}")

Evaluation Metrics for 72h Horizon
RMSE: 264.12
MAE:  136.87


###  CNN-GRU-LSTM Model Evaluation Results ( First Hybrid Model)

| Forecast Horizon | RMSE   | MAE    |
|------------------|--------|--------|
| 12 hours         | 298.16 | 147.36 |
| 24 hours         | 225.50 | 112.40 |
| 48 hours         | 272.79 | 142.87 |
| 72 hours         | 264.12 | 136.87 |
